In [111]:
import pandas as pd
import statistics
from scipy.stats import iqr

info1_file = '/Users/m221138/RA_plasma_metabolites/data/DMARD_prescription_profile_v2.csv'
info2_file = '/Users/m221138/RA_plasma_metabolites/data/RA_biobank_2019June17_sheet1.csv'
data_file = '/Users/m221138/RA_plasma_metabolites/data/Rheum_Biobank_2_Visits.csv'

In [95]:
info1_df = pd.read_csv(info1_file)
info2_df = pd.read_csv(info2_file)
data_df = pd.read_csv(data_file)

patient_info_dict = {}
patientID_list = []
patientID_info_dict = {}

In [96]:
r, c = info1_df.shape
for i in range(r):
    patientID = info1_df.iloc[i,0]
    
    if patientID not in patientID_list:
        patientID_list.append(patientID)
    visit = info1_df.iloc[i,1]
    mtx = info1_df.iloc[i,2]
    pred = info1_df.iloc[i,17]
    patient_info_dict[patientID, visit,"mtx"] = mtx
    patient_info_dict[patientID, visit,"pred"] = pred

In [97]:
print (len(patientID_list))
print (patientID_list)

64
['ST-00010415', 'ST-00010421', 'ST-00011458', 'ST-00044073', 'ST-00044885', 'ST-00045433', 'ST-00057286', 'ST-00057532', 'ST-00060623', 'ST-00062958', 'ST-00075023', 'ST-00128432', 'ST-00177454', 'ST-00242187', 'ST-00286016', 'ST-00286572', 'ST-00287578', 'ST-00295414', 'ST-00319255', 'ST-00320227', 'ST-00339988', 'ST-00360236', 'ST-00418342', 'ST-00423444', 'ST-00466055', 'ST-00505289', 'ST-00525539', 'ST-00531850', 'ST-00550067', 'ST-00596988', 'ST-00597582', 'ST-00606222', 'ST-00611296', 'ST-00616972', 'ST-00622319', 'ST-00624870', 'ST-00630737', 'ST-00631493', 'ST-00632345', 'ST-00634998', 'ST-00641613', 'ST-00648002', 'ST-00651926', 'ST-00652037', 'ST-00653378', 'ST-00656708', 'ST-00659767', 'ST-00660767', 'ST-00685031', 'ST-00705243', 'ST-00705245', 'ST-00708220', 'ST-00709819', 'ST-00711931', 'ST-00714687', 'ST-00718962', 'ST-00723045', 'ST-00730777', 'ST-00731125', 'ST-00731336', 'ST-00731705', 'ST-00732959', 'ST-00739471', 'ST-00745902']


In [98]:
info2_df.head()
r,c = info2_df.shape
mcID_list = []

for i in range(r):
    patientID = info2_df.iloc[i,0]
#     print (patientID)
    mcID = info2_df.iloc[i,2]
    
    if patientID in patientID_list:
        if mcID not in mcID_list:
            patientID_info_dict[patientID] = mcID
            mcID_list.append(mcID)

In [58]:
data_df.head()
print (len(mcID_list))
print (len(patientID_list))
print (mcID_list)

64
64
[2715364, 3025155, 2531748, 1952584, 1728881, 2953603, 2614934, 3709490, 3138082, 1751310, 2536175, 2509601, 3802693, 3430633, 1331643, 1903879, 2464972, 6340182, 4078806, 6397929, 3540375, 4364050, 7176849, 3609406, 3398103, 3425189, 2906656, 3984153, 1806113, 3430782, 6426164, 1890468, 2228154, 8347225, 7074390, 7181912, 3677416, 7318316, 5336919, 7441199, 7047170, 6481216, 9893262, 2227714, 4274053, 6084011, 3518596, 2418423, 6136285, 5435589, 7432029, 6125569, 6202830, 3384920, 1867679, 5380366, 8053229, 6306559, 7020550, 6134399, 3008126, 6112046, 6466177, 7450086]


In [99]:
data_df.head()
r,c = data_df.shape
for i in range(r):
    mcID = data_df.iloc[i,1]
    try: mcID = int(mcID)
    except ValueError: mcID = 0
    mtx_dose_mg = data_df['Dose of methotrexate: mg/week (00.0 to 25.0)'][i]
    pred_dose_mg = data_df['Current prednisone dose - mg/day (0 to 100)'][i]
    visit = data_df['Visit Description'][i]

    try:
        if '1' in visit:
            visit_status = 1
        if '2' in visit:
            visit_status = 2

        patient_info_dict[mcID, visit_status,"met"] = mtx_dose_mg
        patient_info_dict[mcID, visit_status,"pred"] = pred_dose_mg
    except TypeError:
        None

In [117]:
# patient_info_dict[patientID, visit,"mtx"] = mtx
# patient_info_dict[patientID, visit,"pred"] = pred
# patientID_info_dict[mcID, "met"] = mtx_dose_mg
mtx1_list = []
mtx2_list = []
pred1_list = []
pred2_list = []

for patientID in patientID_list:
#     print (patientID)
#     print (patientID_info_dict)
    mcID = patientID_info_dict[patientID]

    #visit 1
    mtx_status = patient_info_dict[patientID, 1, "mtx"]
    pred_status = patient_info_dict[patientID, 1, "pred"]
    mtx_dose_mg = patient_info_dict[mcID, 1, "met"]
    pred_dose_mg = patient_info_dict[mcID, 1, "met"]

    if mtx_status == "yes":
        mtx1_list.append(mtx_dose_mg)
    if pred_status == "yes":
        pred1_list.append(pred_dose_mg)
    
    #visit2
    mtx_status = patient_info_dict[patientID, 2, "mtx"]
    pred_status = patient_info_dict[patientID, 2, "pred"]
    mtx_dose_mg = patient_info_dict[mcID, 2, "met"]
    pred_dose_mg = patient_info_dict[mcID, 2, "met"]
    
    if mtx_status == "yes":
        mtx2_list.append(mtx_dose_mg)
    if pred_status == "yes":
        pred2_list.append(pred_dose_mg)
        
print ("MTX visit 1")
print (len(mtx1_list))
print (statistics.median(mtx1_list))
print(iqr(mtx1_list, rng=(25,75), interpolation='midpoint'))
print ("MTX visit 2")
print (len(mtx2_list))
print (statistics.median(mtx2_list))
print(iqr(mtx2_list, rng=(25,75), interpolation='midpoint'))
print ("PRED visit 1")
print (len(pred1_list))
print (pred1_list)
print (statistics.median(pred1_list))
print(iqr(pred1_list, rng=(25,75), interpolation='midpoint'))
print ("PRED visit 2")
print (len(pred2_list))
print (pred2_list)
print (statistics.median(pred2_list))
print (iqr(pred2_list, rng=(25,75), interpolation='midpoint'))


MTX visit 1
48
20.0
10.0
MTX visit 2
49
20.0
10.0
PRED visit 1
29
[20.0, 25.0, 25.0, 10.0, 17.5, 25.0, 12.5, nan, nan, nan, 20.0, nan, 25.0, 25.0, 20.0, 15.0, 22.5, 25.0, nan, 22.5, 15.0, 10.0, 22.5, 15.0, 20.0, nan, 25.0, 20.0, 25.0]
22.5
nan
PRED visit 2
28
[20.0, 25.0, 15.0, 17.5, 25.0, 12.5, 22.5, nan, nan, 25.0, nan, 25.0, nan, 25.0, 25.0, 17.5, 20.0, 25.0, nan, 25.0, 15.0, 10.0, 17.5, 25.0, 20.0, 12.5, 25.0, 25.0]
22.5
nan
